# Assignment 2: Sequence to Sequence Natural Language to SQL

Lucas Ortiz Duhart
lo21410

## Imports

In [127]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from keras.models import Model
from keras.layers import Input, LSTM,   Dense

## Data Preprocessing

In [128]:
df = pd.read_json('https://raw.githubusercontent.com/Lucothron/CE888-DataScience/main/CE888-master/CE888-master/Assignment%202/train_spider.json')

df.shape

(7000, 7)

In [129]:
df.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [157]:
# Select only queries that include 'avg', 'min', 'max', or 'count' as they are more mathematically inclined

count = 0
data = []
for row in df.itertuples(index = False):
  if 'avg' in row[3] or 'min' in row[3] or 'max' in row[3] or 'count' in row[3]:
    count += 1
    data.append(row)
    #print(row[3])

df_new = pd.DataFrame(data)
df_clean = df_new.drop(['db_id', 'query_toks', 'query_toks_no_value', 'question_toks', 'sql'], axis = 1)

num_pairs = df_clean.shape[0]

df_clean.head()

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
2,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
3,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...
4,SELECT creation FROM department GROUP BY creat...,In which year were most departments established?


In [158]:
cont = 0

new_data = []
for index, value in df_clean['query'].items():
  new_val = '$ ' + value + ' @'
  new_data.append(new_val)

start_end_data = pd.Series(new_data)
df_start_end = df_clean.assign(query_chars = start_end_data)

In [159]:
df_chars = df_start_end.drop(['query'], axis=1)

df_chars.head()

,question,query_chars
0,How many heads of the departments are older th...,$ SELECT count(*) FROM head WHERE age > 56 @
1,What are the maximum and minimum budget of the...,"$ SELECT max(budget_in_billions) , min(budget..."
2,What is the average number of employees of the...,$ SELECT avg(num_employees) FROM department WH...
3,What are the names of the states where at leas...,$ SELECT born_state FROM head GROUP BY born_st...
4,In which year were most departments established?,$ SELECT creation FROM department GROUP BY cre...


In [164]:
def numChars(data):
  chars = []
  charCount = 0

  for index, line in data.items():
    for char in line:
      if char not in chars:
        chars.append(char)
        charCount += 1

  return charCount, chars

In [165]:
def maxLength(data):
  max = 0

  for index, line in data.items():
    temp = len(line)
    if temp > max:
      max = temp
  
  return max

In [166]:
num_eng_char, input_chars = numChars(df_chars['question'])
num_sql_char, target_chars = numChars(df_chars['query_chars'])

max_eng_sent = maxLength(df_chars['question'])
max_sql_sent = maxLength(df_chars['query_chars'])

print(num_pairs, max_eng_sent, num_eng_char)
print(num_pairs, max_sql_sent, num_sql_char)

3059 224 73
3059 406 84


In [167]:
inp_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

In [168]:
encode_inp_data = np.zeros(
    (num_pairs, max_eng_sent, num_eng_char), dtype='float32')
decode_inp_data = np.zeros(
    (num_pairs, max_sql_sent, num_sql_char), dtype='float32')
encode_inp_data = np.zeros(
    (num_pairs, max_sql_sent, num_sql_char), dtype='float32')

In [ ]:
for i, 